<a href="https://colab.research.google.com/github/maximuu19/DCGAN_with_AMP/blob/master/forecasting_model_cnn_neuralforecast%20%F0%9F%93%88%F0%9F%93%88%F0%9F%93%8A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install neuralforecast


In [ ]:
# !jupyter notebook --NotebookApp.iopub_msg_rate_limit=99999

# import libraries

In [ ]:
# import libs for models
import pandas as pd
import numpy as np


from neuralforecast.models.timesnet import TimesNet
from neuralforecast.losses.pytorch import MAPE

from ray import tune

from neuralforecast import NeuralForecast
from neuralforecast.auto import AutoNHITS, AutoLSTM, AutoTimesNet

import warnings
warnings.filterwarnings('ignore')


# read dataframe

In [ ]:
df_4800 = pd.read_csv(r"/content/df_4800_winsorized.csv")
df_4800.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
df_4800['ds'] = pd.to_datetime(df_4800['ds'])
df_4800.info()
display(df_4800.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1367 entries, 0 to 1366
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   y          1367 non-null   float64       
 1   unique_id  1367 non-null   int64         
 2   ds         1367 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 32.2 KB


,y,unique_id,ds
0,41.00,1,2020-04-02
1,40.34,1,2020-04-03
2,40.34,1,2020-04-04


In [ ]:
df_4800 = df_4800[['unique_id', 'ds', 'y']] # re-order

In [ ]:
import torch
torch.cuda.is_available()

True

# split data

In [ ]:
def df_summ(df, h=2, t=2):
    display(pd.concat([df.head(h), df.tail(h)], axis=0))
    print('len ', df.shape[0], '\n\n')

def split_df(df_4800, n: int) -> list:

    train = df_4800.head(n*(-1))
    test = df_4800.tail(n)
    df_summ(train)
    df_summ(train)
    return [train, test]


horizon = 60
[train, test] = split_df(df_4800, n = horizon)

,unique_id,ds,y
0,1,2020-04-02,41.00
1,1,2020-04-03,40.34
1305,1,2023-10-29,88.64
1306,1,2023-10-30,88.64


len  1307 




,unique_id,ds,y
0,1,2020-04-02,41.00
1,1,2020-04-03,40.34
1305,1,2023-10-29,88.64
1306,1,2023-10-30,88.64


len  1307 




# train models

In [ ]:
from neuralforecast.models import NBEATS, NHITS, TimesNet, iTransformer, NBEATSx, PatchTST, DeepAR
from neuralforecast.losses.pytorch import MAPE


seed = 1
horizon = 60
input_size = 2 * horizon
loss=MAPE()
# batch_size = 32
models = [
          NBEATS(input_size=input_size,        h=horizon, loss=loss, random_seed=seed),
          NHITS(input_size=input_size,         h=horizon, loss=loss, random_seed=seed),
          TimesNet(input_size=input_size,      h=horizon, loss=loss, random_seed=seed), # slow train
          iTransformer(input_size= input_size, h=horizon, loss=loss, random_seed=seed, n_series=2),
          # DeepAR(input_size= input_size,       h=horizon,            random_seed=seed, batch_size=batch_size),
          # PatchTST( input_size= input_size,    h=horizon, loss=loss, random_seed=seed, batch_size=batch_size)
          ]

nf = NeuralForecast(models=models, freq='D')


INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
nf.fit(df=train, verbose=False, val_size=horizon)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MAPE          | 0     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 2.8 M 
-----------------------------------------------
2.8 M     Trainable params
21.8 K    Non-trainable params
2.8 M     Total params
11.152    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


Results on test data

In [ ]:
forecasted = nf.predict()
forecasted

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

,ds,NBEATS,NHITS,TimesNet,iTransformer
unique_id,,,,,
1,2023-10-31,88.934013,88.990662,85.693611,88.862251
1,2023-11-01,89.344612,88.130585,85.603706,88.782333
1,2023-11-02,89.190216,88.578201,87.146408,88.708084
1,2023-11-03,91.365685,90.582466,88.551445,89.939468
1,2023-11-04,91.693779,91.748070,86.284592,89.006332
1,2023-11-05,92.562691,92.705460,86.492271,89.423889
1,2023-11-06,92.152107,93.058609,87.471832,89.065079
1,2023-11-07,92.311211,93.925812,90.533310,89.690826
1,2023-11-08,93.476295,95.590439,89.013992,89.260826


In [ ]:
forecasted = forecasted[['ds', 'NBEATS', 'NHITS', 'TimesNet', 'iTransformer']]

In [ ]:
import numpy as np

def calculate_mape(actual: list[float], predicted: list[float]) -> float:
    """
    Calculate the Mean Absolute Percentage Error (MAPE)
    """
    actual, predicted = np.array(actual), np.array(predicted)

    # Check if actual array contains zero.
    if np.any(actual==0):
        return "Error: Actual values array contains zero which will lead to division by zero error."

    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape

In [ ]:
temp = {}
for col in forecasted.columns[1:]:
    p = list(forecasted[col])
    a = list(test['y'])
    temp[col] = calculate_mape(actual=a, predicted=p)

display(pd.Series(temp))

pd.DataFrame(list(temp.items()), columns=['Model', 'MAPE']).to_csv('frc on test data_h_60.csv')


NBEATS          32.840753
NHITS           28.973791
TimesNet        13.190445
iTransformer    16.973210
dtype: float64

**metrics for train data**

In [ ]:
len(train)

1307

nf.insample_predict() is not working

In [ ]:
# simple dataloader for getting predictions on train dataset by feeding it to nf.predict part 1
### Even dataloader
all_forecasted = pd.DataFrame()
times = np.ceil(len(train)/120)
st = 107

for i in range(int(times) - 1):

    curr_df = train.iloc[ 120*i + st: 120*(i+1) + st ]

    forecasted = nf.predict(df=curr_df, verbose=False) # forecasting on 120 days before training data

    all_forecasted = pd.concat([all_forecasted, forecasted])


    print(i, '\n\n')
    display(pd.concat([curr_df.head(1), curr_df.tail(1)]))
    print('len cur ', len(curr_df), '\n')

    display(pd.concat([forecasted.head(1), forecasted.tail(1)]))
    print('len frc ', len(forecasted), '\n')
    print('*'*10, '\n\n')
#     227,   38.86,  1,   2020-11-15  <- We are getting forecasted data from 2020/11/15 to end of tesst

frc1 = all_forecasted
del all_forecasted


In [ ]:
frc1 = frc1[['ds', 'NBEATS', 'NHITS', 'TimesNet', 'iTransformer']]

In [ ]:
# simple dataloader for getting predictions on train dataset by feeding it to nf.predict part 2

### Odd dataloader

all_forecasted = pd.DataFrame()
times = np.ceil(len(train)/120)
st = 107 - 60

for i in range(int(times) - 1 ):

    curr_df = train.iloc[ 120*i + st: 120*(i+1) + st ]


    forecasted = nf.predict(df=curr_df) # forecasting on 120 days before training data

    all_forecasted = pd.concat([all_forecasted, forecasted])


    print(i, '\n')
    display('cur df', pd.concat([curr_df.head(2), curr_df.tail(2)]))
    print('len cur ', len(curr_df), '\n')

    display('frc df', pd.concat([forecasted.head(1), forecasted.tail(1)]))
    print('len frc ', len(forecasted), '\n')
    print('*'*10, '\n\n')


frc2 = all_forecasted
del all_forecasted


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

,unique_id,ds,y
1127,1,2023-05-04,91.28
1128,1,2023-05-05,87.26
1245,1,2023-08-30,82.54
1246,1,2023-08-31,82.54


len cur  120 



'frc df'

,ds,NBEATS,NHITS,TimesNet,iTransformer
unique_id,,,,,
1,2023-09-01,82.162392,82.603340,73.772713,83.183594
1,2023-10-30,88.202583,93.406494,75.708580,93.910858


len frc  60 

********** 




In [ ]:
frc2 = frc2[['ds', 'NBEATS', 'NHITS', 'TimesNet', 'iTransformer']]

In [ ]:
display('frc1 df', pd.concat([frc1.head(2), frc1.tail(2)]))
print('len frc1 ', len(frc1), '\n')

'frc1 df'

,ds,NBEATS,NHITS,TimesNet,iTransformer
unique_id,,,,,
1,2020-11-15,38.874844,38.834553,34.911049,36.655148
1,2020-11-16,38.938530,38.871284,34.716789,36.689545
1,2023-12-28,114.462349,102.662468,81.133583,86.708656
1,2023-12-29,114.130608,102.698746,82.032295,86.277237


len frc1  600 



In [ ]:
display('frc2 df', pd.concat([frc2.head(2), frc2.tail(2)]))
print('len frc2 ', len(frc2), '\n')

'frc2 df'

,ds,NBEATS,NHITS,TimesNet,iTransformer
unique_id,,,,,
1,2020-09-16,33.067425,33.175694,32.972282,33.404831
1,2020-09-17,33.176678,33.177391,32.865395,33.548958
1,2023-10-29,87.511009,93.166260,76.034630,93.904327
1,2023-10-30,88.202583,93.406494,75.708580,93.910858


len frc2  600 



In [ ]:
merged1 = train.merge(frc1, on='ds', how='left')
merged1

,unique_id,ds,y,NBEATS,NHITS,TimesNet,iTransformer
0,1,2020-04-02,41.00,NaN,NaN,NaN,NaN
1,1,2020-04-03,40.34,NaN,NaN,NaN,NaN
2,1,2020-04-04,40.34,NaN,NaN,NaN,NaN
3,1,2020-04-05,40.34,NaN,NaN,NaN,NaN
4,1,2020-04-06,40.34,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1302,1,2023-10-26,94.45,NaN,NaN,NaN,NaN
1303,1,2023-10-27,88.64,NaN,NaN,NaN,NaN
1304,1,2023-10-28,88.64,NaN,NaN,NaN,NaN
1305,1,2023-10-29,88.64,NaN,NaN,NaN,NaN


In [ ]:
merged_2 = train.merge(frc2, on='ds', how='left')
merged_2.columns[-6:]

Index(['ds', 'y', 'NBEATS', 'NHITS', 'TimesNet', 'iTransformer'], dtype='object')

In [ ]:
merged_2.iloc[:, -4:]

,NBEATS,NHITS,TimesNet,iTransformer
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
1302,85.959991,90.094566,77.990829,93.195312
1303,88.239372,90.923683,75.856644,93.242844
1304,85.915108,92.680557,76.390297,93.636467
1305,87.511009,93.166260,76.034630,93.904327


In [ ]:

frc_on_train = (merged_2.iloc[:, -4:].fillna(0) + merged1.iloc[:, -4:].fillna(0)).iloc[167: , : ]
frc_on_train

,NBEATS,NHITS,TimesNet,iTransformer
167,33.067425,33.175694,32.972282,33.404831
168,33.176678,33.177391,32.865395,33.548958
169,33.383209,33.361549,33.096722,33.547134
170,33.665745,33.582684,31.987942,33.353149
171,34.169544,33.901459,32.675270,33.650482
...,...,...,...,...
1302,85.959991,90.094566,77.990829,93.195312
1303,88.239372,90.923683,75.856644,93.242844
1304,85.915108,92.680557,76.390297,93.636467
1305,87.511009,93.166260,76.034630,93.904327


In [ ]:
temp = {}
for col in frc_on_train.columns:
    p = list(frc_on_train[col])
    a = list(merged_2.iloc[167:,:]['y'])
    temp[col] = calculate_mape(actual=a, predicted=p)

display(pd.Series(temp))

NBEATS           1.503776
NHITS            2.000858
TimesNet        14.998446
iTransformer     9.750561
dtype: float64

In [ ]:
pd.DataFrame(list(temp.items()), columns=['Model', 'MAPE']).to_csv('frc on train.csv')


In [ ]:
# from neuralforecast.auto import AutoNHITS

# nhits_config = AutoNHITS.get_default_config(h = 12, backend="ray")
# print(nhits_config)

# Saving

In [ ]:
nf.save(path='./checkpoints/NF/',
        overwrite=True,
        save_dataset=True,)

In [ ]:
! zip -r all_files.zip *

# ARIMA

In [ ]:
%%capture
!pip install pmdarima

In [ ]:
ar_train = train.set_index('ds').drop(columns=['unique_id'])
ar_test = test.set_index('ds').drop(columns=['unique_id'])
display(ar_test)

In [ ]:
ar_train['month'] = ar_train.index.month
ar_test['month'] = ar_test.index.month
ar_test

In [ ]:
import pandas as pd
import pmdarima as pm
from pmdarima.model_selection import train_test_split

model = pm.auto_arima(ar_train[['y']], exogenous=ar_train[['month']],
                      start_p = 1, start_q = 1,
                      test='adf',
                      max_p = 3, max_q = 3, m = 30,
                      start_P = 0, seasonal = True,
                      d = 1, D = 1, trace = True,
                     error_action='ignore',
                           suppress_warnings=True,
                           stepwise=True,)
model.summary()

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[30]             : AIC=inf, Time=37.72 sec
 ARIMA(0,1,0)(0,1,0)[30]             : AIC=7685.510, Time=0.52 sec
 ARIMA(1,1,0)(1,1,0)[30]             : AIC=7332.373, Time=3.93 sec
 ARIMA(0,1,1)(0,1,1)[30]             : AIC=inf, Time=13.21 sec
 ARIMA(1,1,0)(0,1,0)[30]             : AIC=7687.510, Time=0.33 sec
 ARIMA(1,1,0)(2,1,0)[30]             : AIC=7190.905, Time=6.62 sec
 ARIMA(1,1,0)(2,1,1)[30]             : AIC=inf, Time=64.96 sec
 ARIMA(1,1,0)(1,1,1)[30]             : AIC=inf, Time=19.65 sec
 ARIMA(0,1,0)(2,1,0)[30]             : AIC=7189.045, Time=6.06 sec
 ARIMA(0,1,0)(1,1,0)[30]             : AIC=7330.373, Time=1.82 sec
 ARIMA(0,1,0)(2,1,1)[30]             : AIC=inf, Time=60.54 sec
 ARIMA(0,1,0)(1,1,1)[30]             : AIC=inf, Time=14.12 sec


In [ ]:
# Print model parameters
print(f"Best model order (p, d, q) = {model.order}")
print(f"Best seasonal order (P, D, Q, m) = {model.seasonal_order}")

In [ ]:
fitted, confint = model.predict(n_periods=60, return_conf_int=True,)

In [ ]:
p = list(fitted.values)

a = list(ar_test['y'])

calculate_mape(a, p) # <-  mape on test

In [ ]:
# 30 then up... for m

# Sarimax

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

get_parametes = model.get_params()
print(type(get_parametes))
get_parametes

In [ ]:
order_aa = get_parametes.get('order')
seasonal_order_aa = get_parametes.get('seasonal_order')
print('order:', order_aa)
print('seasonal_order:', seasonal_order_aa)
print('order DTYPE:', type(order_aa))
print('seasonal_order DTYPE:', type(seasonal_order_aa))

model_ss = SARIMAX(ar_train[['y']], exog=ar_train[['month']],
            order = (order_aa[0], order_aa[1], order_aa[2]),
            seasonal_order =(seasonal_order_aa[0],
seasonal_order_aa[1], seasonal_order_aa[2], seasonal_order_aa[3]))

result_ss = model_ss.fit()
result_ss.summary()

In [ ]:
predictions = result_ss.predict(start=len(ar_train), end=len(ar_train) + len(ar_test)-1, exog=ar_test[['month']])


In [ ]:
p = list(predictions.values)

a = list(ar_test['y'])

calculate_mape(a, p) # <-  mape on test

###### d = None

In [ ]:
import pandas as pd
import pmdarima as pm
from pmdarima.model_selection import train_test_split

model = pm.auto_arima(ar_train[['y']], exogenous=ar_train[['month']],
                      start_p = 1, start_q = 1,
                      test='adf',
                      max_p = 3, max_q = 3, m = 30,
                      start_P = 0, seasonal = True,
                      d = None, D = 1, trace = True,
                     error_action='ignore',
                           suppress_warnings=True,
                           stepwise=True,)
model.summary()


fitted, confint = model.predict(n_periods=60, return_conf_int=True,)


In [ ]:
# Print model parameters
print(f"Best model order (p, d, q) = {model.order}")
print(f"Best seasonal order (P, D, Q, m) = {model.seasonal_order}")

In [ ]:
p = list(fitted.values)

a = list(ar_test['y'])

calculate_mape(a, p) # <-  mape on test

##### Neuralprophet

In [ ]:
# %%capture
# ! pip install neuralprophet


In [ ]:
# !zip -r all_files.zip *
# !zip -r files.zip *

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os
from google.colab import files
import itertools

In [ ]:
# Import libraries
import os
import shutil
from google.colab import files

# Create a zip file
zip_name = '/content/all_files.zip'
directory_to_zip = '/content/checkpoints'

# Zip the directory
shutil.make_archive(zip_name, 'zip', directory_to_zip, '.')

# Download the zip file
files.download(zip_name)